In [1]:
import pandas as pd
import numpy as np
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [2]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)


In [3]:
def create_sequences(data, seq_length, feature_cols, target_col):
    xs, ys = [], []
    for _, group in data.groupby('driverId'):
        group = group.sort_values(['year', 'round'])
        features = group[feature_cols].values
        targets = group[target_col].values
        
        if len(group) <= seq_length:
            continue
            
        for i in range(len(group) - seq_length):
            xs.append(features[i:(i + seq_length)])
            ys.append(targets[i + seq_length])
            
    return np.array(xs), np.array(ys)


In [4]:
class F1Dataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).view(-1, 1)
        
    def __len__(self): return len(self.x)
    def __getitem__(self, idx): return self.x[idx], self.y[idx]

In [5]:
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_dim, 1) # v1과 동일하게 tanh 제거

    def forward(self, lstm_output):
        attn_weights = torch.softmax(self.attn(lstm_output), dim=1)
        context = torch.sum(attn_weights * lstm_output, dim=1)
        return context, attn_weights


In [6]:
class F1SprintPredictor(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=2):
        super(F1SprintPredictor, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=0.2)
        self.attention = Attention(hidden_dim)
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 1) 
        )

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        context, _ = self.attention(lstm_out)
        return self.fc(context)


In [7]:
def train_and_evaluate_v2():
    
    file_path = '../data_engineered/modern_train_ready_v2.csv'
    if not os.path.exists(file_path):
        print("Error: No v2 preprocessed file found.")
        return

    df = pd.read_csv(file_path)
    
    
    FEATURE_COLS = [
        'grid', 'qual_pos', 'q_rel_gap', 'age_at_race', 
        'driver_form', 'team_form', 'driver_experience', 'circuit_avg_pos'
    ]
    TARGET_COL = 'sprint_target'
    SEQ_LENGTH = 3
    
    
    X, y = create_sequences(df, SEQ_LENGTH, FEATURE_COLS, TARGET_COL)
    
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    train_loader = DataLoader(F1Dataset(X_train, y_train), batch_size=16, shuffle=True)
    test_loader = DataLoader(F1Dataset(X_test, y_test), batch_size=1)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = F1SprintPredictor(input_dim=len(FEATURE_COLS), hidden_dim=64).to(device)
    
    criterion_mse = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001) 

    
    EPOCHS = 100
    print(f"v2 Model train start (Architecture identical to v1)...")
    
    for epoch in range(EPOCHS):
        model.train()
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion_mse(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
        if (epoch + 1) % 20 == 0:
            print(f"Epoch [{epoch+1:3d}/{EPOCHS}] Complete")

    
    model.eval()
    all_preds, all_actuals = [], []
    
    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x = batch_x.to(device)
            preds = model(batch_x)
            all_preds.append(preds.item())
            all_actuals.append(batch_y.item())

    mae = mean_absolute_error(all_actuals, all_preds)
    mse = mean_squared_error(all_actuals, all_preds)
    rmse = np.sqrt(mse)

    print("\n" + "="*50)
    print("--- v2 Final evaluate report (Architecture Consistent) ---")
    print(f"1. MAE  : {mae:.4f}")
    print(f"2. MSE  : {mse:.4f}")
    print(f"3. RMSE : {rmse:.4f}")
    print("="*50)

In [8]:
if __name__ == "__main__":
    train_and_evaluate_v2()

v2 Model train start (Architecture identical to v1)...
Epoch [ 20/100] Complete
Epoch [ 40/100] Complete
Epoch [ 60/100] Complete
Epoch [ 80/100] Complete
Epoch [100/100] Complete

--- v2 Final evaluate report (Architecture Consistent) ---
1. MAE  : 3.0038
2. MSE  : 13.9592
3. RMSE : 3.7362
